In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import requests
from bs4 import BeautifulSoup
import csv
import winsound as ws

In [2]:
def getStockCode(market):
    if market == 'kosdaq':
        url_market = 'kosdaqMkt'
    elif market == 'kospi':
        url_market = 'stockMkt'
    else:
        print('invalid market ')
        return
    url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=%s' % url_market
    df = pd.read_html(url, header=0)[0]
    return df
if __name__ == '__main__':
    result_df = getStockCode('kospi')
    #print(result_df)


In [9]:
cods = getStockCode('kosdaq')
codsY = cods[['회사명','종목코드']]
codsY['종목코드']=codsY['종목코드'].astype(str)
codsY['종목코드']=codsY['종목코드'].str.zfill(6)
#codsY['종목코드']=codsY['종목코드'].str.zfill(6) + '.KS'
#samsung = data.get_data_yahoo(codsY[codsY['회사명']=='현대자동차']['종목코드'].iloc[0])
#samsung.tail()
#print(codsY['종목코드'])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [10]:
def get_bs_obj(company_code):
    url = "https://finance.naver.com/item/main.nhn?code=" + company_code
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content,"html.parser")
    return bs_obj

def get_chart(company_code):
    print(company_code)
    temp=[]
    bs_obj = get_bs_obj(company_code)
    td_first = bs_obj.find("td",{"class":"first"})
    if(td_first) == None:
        return temp
    else:
        blind = td_first.find("span",{"class":"blind"})    
    
    #회사명
    comp_name = bs_obj.find("div",{"class":"wrap_company"})
    comp_name_h2 = comp_name.find("h2")
    temp.append(comp_name_h2.text)
    
    ##시가총액
    con1 = bs_obj.find("div",{"class":"tab_con1"})
    con1_td = con1.findAll("td")
    
    stocks = con1_td[2].find("em")
    if stocks == None:
        temp.append('')
    else:
        temp.append(stocks.text) 
    if len(con1_td) <= 3:
        temp.append('')
    else:
        stock_price = con1_td[3].find("em")
        temp.append(stock_price.text)

        
    #업종
    section_name = bs_obj.find("div",{"class":"section trade_compare"})
    if section_name == None:
        temp.append('')
    else:
        section_name_h4 = section_name.find("h4")
        em_4 = section_name_h4.find("em")
        a_4 = em_4.find("a")
        temp.append(a_4.text)
        
    temp.append("'" + company_code)
        
    #현재가
    table = bs_obj.find("div", {"class":"today"})
    table2 = table.find("p",{"class":"no_today"})
    if table2 == None:
        temp.append('')
    else:
        now = table2.find("span",{"class":"blind"})
        if now == None:
            temp.append('')
        else:
            temp.append(now.text)
        
    #종가(전일)
    close = blind.text
    temp.append(close)
        
    #고가,저가,시가    
    table = bs_obj.find("table", {"class":"no_info"})
    trs = table.findAll("tr")
    first_tr = trs[0]
    first_tr_tds = first_tr.findAll("td")
    first_tr_tds_second = first_tr_tds[1]
    second_tr = trs[1]
    second_tr_tds = second_tr.findAll("td")
    second_tr_tds_first = second_tr_tds[0]
    second_tr_tds_second = second_tr_tds[1]
    high = first_tr_tds_second.find("span",{"class":"blind"})
    low = second_tr_tds_first.find("span",{"class":"blind"})
    now = second_tr_tds_second.find("span",{"class":"blind"})
    temp.append(high.text)
    temp.append(low.text)
    temp.append(now.text)
        
    ##투자정보
    #PER
    table_invest = bs_obj.find("table",{"class":"per_table"})
    if table_invest == None:
        return temp
    tbodys = table_invest.findAll("tbody")
    tbody_1 = tbodys[0]
    em= tbody_1.find("em")
    temp.append(em.text)
    #추정PER
    em_1 = table_invest.find("em",{"id":"_cns_per"})
    if em_1 == None:
        temp.append('')
    else:
        temp.append(em_1.text) 
    #PBR
    em_2 = table_invest.find("em",{"id":"_pbr"})
    if em_2 == None:
        temp.append('')
    else:
        temp.append(em_2.text)
    #동일업종PER
    table_invest_2 = bs_obj.find("table",{"summary":"동일업종 PER 정보"})
    if table_invest_2 == None:
        temp.append('')
    else:
        em_3 = table_invest_2.find("em")
        if em_3 == None:
            temp.append('')
        else:
            temp.append(em_3.text)
        
    #재무제표 분석
    table_finance_statement = bs_obj.find("table", {"class":"tb_type1 tb_num tb_type1_ifrs"})
    if table_finance_statement == None:
        return temp
    tbody = table_finance_statement.find("tbody")
    trs = tbody.findAll("tr")
    tr_1 = trs[0] #매출액(억원)
    tr_1_tds = tr_1.findAll("td")
    if len(tr_1_tds)==1:
        return temp
    for i in range(10):
        temp.append(tr_1_tds[i].text.strip())
    tr_2 = trs[1] #영업이익(억원)
    tr_2_tds = tr_2.findAll("td")
    for i in range(10):
        temp.append(tr_2_tds[i].text.strip())
    tr_3 = trs[2] #당기순이익(억원)
    tr_3_tds = tr_3.findAll("td")
    for i in range(10):
        temp.append(tr_3_tds[i].text.strip())
    tr_4 = trs[3] #영업이익률(%)
    tr_4_tds = tr_4.findAll("td")
    for i in range(10):
        temp.append(tr_4_tds[i].text.strip())
    tr_5 = trs[5] #ROE(%)
    tr_5_tds = tr_5.findAll("td")
    for i in range(10):
        temp.append(tr_5_tds[i].text.strip())
    tr_6 = trs[6] #영업이익률(%)
    tr_6_tds = tr_6.findAll("td")
    for i in range(10):
        temp.append(tr_6_tds[i].text.strip())
    tr_7 = trs[9] #EPS(원)
    tr_7_tds = tr_7.findAll("td")
    for i in range(10):
        temp.append(tr_7_tds[i].text.strip())
    tr_8 = trs[10] #PER(배)
    tr_8_tds = tr_8.findAll("td")
    for i in range(10):
        temp.append(tr_8_tds[i].text.strip())
    tr_9 = trs[11] #BPS(원)
    tr_9_tds = tr_9.findAll("td")
    for i in range(10):
        temp.append(tr_9_tds[i].text.strip())
    tr_10 = trs[12] #PBR(배)
    tr_10_tds = tr_10.findAll("td")
    for i in range(10):
        temp.append(tr_10_tds[i].text.strip())   
      
    return temp
    
####################### Main #######################
#company_codes = ["005930","000660","005680"]
#company_codes = ["005930"]
candle_chart_dataframe=[]
for item in codsY['종목코드']:
#for item in company_codes:
   # price = get_price(b)
    candle_chart_data = get_chart(item)
    candle_chart_dataframe.append(candle_chart_data)

# CSV 파일 출력
with open('200821_naver_finance_kosdaq.csv','w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['회사명','주식수','액면가','업종명','회사코드','현재가','전날종가','고가','저가','시가','PER','추정PER','PBR','동일업종PER','17.12매출액(억원)','18.12매출액(억원)','19.12매출액(억원)','20예상매출액(억원)','19.1Q매출액(억원)','19.2Q매출액(억원)','19.3Q매출액(억원)','19.4Q매출액(억원)','20.1Q매출액(억원)','20.2Q예상매출액(억원)'
                     ,'17.12영업이익(억원)','18.12영업이익(억원)','19.12영업이익(억원)','20예상영업이익(억원)','19.1Q영업이익(억원)','19.2영업이익(억원)','19.3Q영업이익(억원)','19.4Q영업이익(억원)','20.1Q영업이익(억원)','20.2Q예상영업이익(억원)'
                     ,'17.12당기순이익(억원)','18.12당기순이익(억원)','19.12당기순이익(억원)','20예상당기순이익(억원)','19.1Q당기순이익(억원)','19.2Q당기순이익(억원)','19.3Q당기순이익(억원)','19.4Q당기순이익(억원)','20.1Q당기순이익(억원)','20.2Q예상당기순이익(억원)'
                     ,'17.12영업이익률(%)','18.12영업이익률(%)','19.12영업이익률(%)','20예상영업이익률(%)','19.1Q영업이익률(%)','19.2Q영업이익률(%)','19.3Q영업이익률(%)','19.4Q영업이익률(%)','20.1Q영업이익률(%)','20.2Q예상영업이익률(%)'
                     ,'17.12ROE(%)','18.12ROE(%)','19.12ROE(%)','20예상ROE(%)','19.1QROE(%)','19.2QROE(%)','19.2QROE(%)','19.3QROE(%)','20.1QROE(%)','20.2Q예상ROE(%)'
                     ,'17.12부채비율(%)','18.12부채비율(%)','19.12부채비율(%)','20예상부채비율(%)','19.1Q부채비율(%)','19.2Q부채비율(%)','19.3Q부채비율(%)','19.4Q부채비율(%)','20.1Q부채비율(%)','20.2Q예상부채비율(%)'
                     ,'17.12EPS(원)','18.12EPS(원)','19.12EPS(원)','20예상EPS(원)','19.1QEPS(원)','19.2QEPS(원)','19.3QEPS(원)','19.4QEPS(원)','20.1QEPS(원)','20.2Q예상EPS(원)'
                     ,'17.12PER(배)','18.12PER(배)','19.12PER(배)','20예상PER(배)','19.1QPER(배)','19.2QPER(배)','19.3QPER(배)','19.4QPER(배)','20.1QPER(배)','20.2Q예상PER(배)'
                     ,'17.12BPS(원)','18.12BPS(원)','19.12BPS(원)','20예상BPS(원)','19.1QBPS(원)','19.2QBPS(원)','19.3QBPS(원)','19.4QBPS(원)','20.1QBPS(원)','20.2Q예상BPS(원)'
                     ,'17.12PBR(배)','18.12PBR(배)','19.12PBR(배)','20예상PBR(배)','19.1QPBR(배)','19.2QPBR(배)','19.3QPBR(배)','19.4QPBR(배)','20.1QPBR(배)','20.2Q예상PBR(배)'
                    ])
    writer.writerows(candle_chart_dataframe)

051500
245620
083450
028150
351340
099520
067290
024120
046440
060720
337450
340350
089230
032540
051390
094480
198440
215000
307280
043650
204020
286750
089600
085910
007390
031390
012340
085670
078140
290120
003310
023910
006620
099410
196490
066900
092070
039840
033130
033310
263020
082800
060240
219420
227950
086900
021880
327260
250060
118990
028040
207760
206640
053030
337930
023600
111870
009300
042940
100660
080470
011560
258830
053450
049180
208370
131100
013810
049830
131090
222800
217330
010280
115480
900120
123860
149950
099190
040910
068940
114810
031310
124500
260660
096610
061040
196170
065420
096630
043340
096690
021080
247540
092870
048830
156100
058630
019590
097520
265560
045060
049480
109080
019210
074600
014190
008290
095270
065950
036090
203450
241690
078070
054930
336060
331380
178780
179900
047560
123570
353810
090850
079950
064290
254120
234920
040420
079370
094970
058420
023440
080220
082270
034940
036930
119850
065060
071850
342550
042040
274090
078650
282880

023410
056080
072770
086250
041520
078020
900110
226360
102710
088290
092130
024810
048530
041020
058450
094820
187420
147830
026040
229000
067000
000440
204840
109820
004650
096240
900310
263700
272110
083550
225430
064820
089010
220260
046070
123410
009730
051780
023160
130740
140860
091700
222110
009520
256630
214270
237750
131390
307160
003380
340120
106190
004590
101680
047080
039010
061250
050090
090710
265520
211270
130500
078590
091340
036540
161570
040300
121440
900070
036190
308100
190510
306620
144510
054050
270870
119860
093640
048470
007720
290670
336570
010170
298540
263800
206560
067990
005160
013120
109860
025900
203650
060570
217620
127120
104460
197140
106520
294140
108490
071280
038060
085370
058470
305090
038290
093520
041920
096640
100030
218150
049950
064520
267790
046310
066410
006910
251630
018290
200780
141000
072950
329560
101000
294630
043710
018680
189860
035890
014620
053060
039310
051980
036830
304100
230980
317770
002800
344050
012790
160980
208640
050890

In [12]:
def get_bs_obj(company_code):
    url = "https://finance.naver.com/item/main.nhn?code=" + company_code
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content,"html.parser")
    return bs_obj

def get_chart(company_code):
    temp=[]
    bs_obj = get_bs_obj(company_code)
    td_first = bs_obj.find("td",{"class":"first"})
    print(td_first)
    blind = td_first.find("span",{"class":"blind"})
    
    #회사명
    comp_name = bs_obj.find("div",{"class":"wrap_company"})
    comp_name_h2 = comp_name.find("h2")
    temp.append(comp_name_h2.text)
    
    ##시가총액
    con1 = bs_obj.find("div",{"class":"tab_con1"})
    con1_td = con1.findAll("td")
    
    stocks = con1_td[2].find("em")
    if stocks == None:
        temp.append('')
    else:
        temp.append(stocks.text)
    
    if len(con1_td) <= 3:
        temp.append('')
    else:
        stock_price = con1_td[3].find("em")
        temp.append(stock_price.text)

    
    
    #업종
    section_name = bs_obj.find("div",{"class":"section trade_compare"})
    if section_name == None:
        temp.append('')
    else:
        section_name_h4 = section_name.find("h4")
        em_4 = section_name_h4.find("em")
        a_4 = em_4.find("a")
        temp.append(a_4.text)
        
    
   
    
    temp.append("'" + company_code)
    
    #현재가
    table = bs_obj.find("div", {"class":"today"})
    table2 = table.find("em",{"class":"no_down"})
    now = table2.find("span",{"class":"blind"})
    temp.append(now.text)
        
    #종가(전일)
    close = blind.text
    temp.append(close)
    
    #고가,저가,시가    
    table = bs_obj.find("table", {"class":"no_info"})
    trs = table.findAll("tr")
    first_tr = trs[0]
    first_tr_tds = first_tr.findAll("td")
    first_tr_tds_second = first_tr_tds[1]
    second_tr = trs[1]
    second_tr_tds = second_tr.findAll("td")
    second_tr_tds_first = second_tr_tds[0]
    second_tr_tds_second = second_tr_tds[1]
    high = first_tr_tds_second.find("span",{"class":"blind"})
    low = second_tr_tds_first.find("span",{"class":"blind"})
    now = second_tr_tds_second.find("span",{"class":"blind"})
    temp.append(high.text)
    temp.append(low.text)
    temp.append(now.text)
    
    ##투자정보
    #PER
    table_invest = bs_obj.find("table",{"class":"per_table"})
    if table_invest == None:
        return temp
    tbodys = table_invest.findAll("tbody")
    tbody_1 = tbodys[0]
    em= tbody_1.find("em")
    temp.append(em.text)
    #추정PER
    em_1 = table_invest.find("em",{"id":"_cns_per"})
    if em_1 == None:
        temp.append('')
    else:
        temp.append(em_1.text) 
    #PBR
    em_2 = table_invest.find("em",{"id":"_pbr"})
    if em_2 == None:
        temp.append('')
    else:
        temp.append(em_2.text)
    #동일업종PER
    table_invest_2 = bs_obj.find("table",{"summary":"동일업종 PER 정보"})
    em_3 = table_invest_2.find("em")
    if em_3 == None:
        temp.append('')
    else:
        temp.append(em_3.text)
    
    
    #재무제표 분석
    table_finance_statement = bs_obj.find("table", {"class":"tb_type1 tb_num tb_type1_ifrs"})
    tbody = table_finance_statement.find("tbody")
    trs = tbody.findAll("tr")
    tr_1 = trs[0] #매출액(억원)
    tr_1_tds = tr_1.findAll("td")
    if len(tr_1_tds)==1:
        return None
    for i in range(10):
        temp.append(tr_1_tds[i].text.strip())
    tr_2 = trs[1] #영업이익(억원)
    tr_2_tds = tr_2.findAll("td")
    for i in range(10):
        temp.append(tr_2_tds[i].text.strip())
    tr_3 = trs[2] #당기순이익(억원)
    tr_3_tds = tr_3.findAll("td")
    for i in range(10):
        temp.append(tr_3_tds[i].text.strip())
    tr_4 = trs[3] #영업이익률(%)
    tr_4_tds = tr_4.findAll("td")
    for i in range(10):
        temp.append(tr_4_tds[i].text.strip())
    tr_5 = trs[5] #ROE(%)
    tr_5_tds = tr_5.findAll("td")
    for i in range(10):
        temp.append(tr_5_tds[i].text.strip())
    tr_6 = trs[6] #영업이익률(%)
    tr_6_tds = tr_6.findAll("td")
    for i in range(10):
        temp.append(tr_6_tds[i].text.strip())
    tr_7 = trs[9] #EPS(원)
    tr_7_tds = tr_7.findAll("td")
    for i in range(10):
        temp.append(tr_7_tds[i].text.strip())
    tr_8 = trs[10] #PER(배)
    tr_8_tds = tr_8.findAll("td")
    for i in range(10):
        temp.append(tr_8_tds[i].text.strip())
    tr_9 = trs[11] #BPS(원)
    tr_9_tds = tr_9.findAll("td")
    for i in range(10):
        temp.append(tr_9_tds[i].text.strip())
    tr_10 = trs[12] #PBR(배)
    tr_10_tds = tr_10.findAll("td")
    for i in range(10):
        temp.append(tr_10_tds[i].text.strip())   
        
    return temp

# Main
#company_codes = ["011070","051900","032640"]
company_codes = ["331920"]
candle_chart_dataframe=[]
#for item in codsY['종목코드'][28:30]:
for item in company_codes:
   # price = get_price(b)
    candle_chart_data = get_chart(item)
    candle_chart_dataframe.append(candle_chart_data)
print(candle_chart_dataframe)

None


AttributeError: 'NoneType' object has no attribute 'find'